In [8]:
import plotly.express as px
from traffic.core import Traffic
from traffic.data.datasets import landing_zurich_2019

In [9]:
def plot_trajs_time(traffic):
    """
    Donne en argument un traffic avec des vols que l'on veut voir évoluer en même temps
    N'affiche que l'intersection entre les 2 temps de vol
    """
    # Making sure to have observations every 1s 
    traffic = traffic.resample("1s").eval()

    if str(type(traffic)) == "<class 'traffic.core.traffic.Traffic'>":
        starts = []
        stops = []
        for f in traffic:
            starts.append(f.start)
            stops.append(f.stop)

        traffic = traffic.between(max(starts), min(stops)).resample(64).eval()
        dataset = traffic.data

    else:
        dataset = traffic.resample(64).data

    dataset["Time"] = dataset["timestamp"].apply(lambda x: x.strftime("%H:%M:%S"))

    px.set_mapbox_access_token(
        "pk.eyJ1IjoiZmlndSIsImEiOiJja2JndjNoaDQwYnl6MnJwM2tzNG9vcHc2In0.Ru9xsbyaKn7BNvIYAwPwUg"
    )

    fig = px.scatter_mapbox(
        dataset,
        lat="latitude",
        lon="longitude",
        animation_frame="Time",
        animation_group="flight_id",
        hover_name="flight_id",
        size="altitude",
    )

    for ids in dataset.flight_id.unique():
        fig.add_traces(
            px.line_mapbox(
                dataset[dataset.flight_id == ids], lat="latitude", lon="longitude"
            ).data[0]
        )

    fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 200

    return fig

In [11]:
# Si l'intersection des temps de vol est nulle, m'affiche rien 
fig = plot_trajs_time(landing_zurich_2019[["AEE5SZ_4449", "AEA76TU_392"]])
fig.show()

C:\Users\kruu\.conda\envs\airtraffic\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.

